# Obtaining the Data

Note that this code needs to be run on a kaggle notebook. The bq_helper library uses kaggle permissions to access google big query.

The dataset in question: https://www.kaggle.com/datasets/bigquery/patents

In [ ]:
# Start by importing the bq_helper module and calling on the specific active_project and dataset_name for the BigQuery dataset.
import bq_helper
import  pandas as pd
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package

patents = bq_helper.BigQueryHelper(active_project="patents-public-data",
                                   dataset_name="patents")

In [ ]:
# View table names under the patents data table
bq_assistant = BigQueryHelper("patents-public-data", "patents")
bq_assistant.list_tables()

In [ ]:
# View the first three rows of the publications data table
df_sample = bq_assistant.head("publications", num_rows=3)
df_sample

In [ ]:
# View information on all columns in the publications data table
bq_assistant.table_schema("publications")

In [ ]:
### Test xample Query
query1 = """
SELECT
  publication_number,
  application_number,
  country_code,
  pct_number,
  family_id,
  title_localized,
  abstract_localized,
  claims_localized,
  description_localized,
  publication_date,
  filing_date,
  inventor,
  assignee,
  ipc
FROM
  `patents-public-data.patents.publications`
LIMIT
  10;
        """

In [ ]:
# this function shows the size of the query on the dataset before it was run
# This is important to check as the dataset is terabyte size.
bq_assistant.estimate_query_size(query1)

In [ ]:
## Query for a dataset of patent text to do NLP modelling on
query_patent_text = """
SELECT
  publication_number,
  application_number,
  title_localized,
  abstract_localized,
  claims_localized,
  description_localized,
FROM
  `patents-public-data.patents.publications`
WHERE
    application_kind = 'A' AND
    publication_date > 20180000
LIMIT
  10000;
  """

This query was big (about 200GB)

In [ ]:
query2018 = """
SELECT
  publication_number,
  application_number,
  country_code,
  publication_date,
  filing_date,
  assignee,
  family_id,
  ipc
FROM
  `patents-public-data.patents.publications`
WHERE
    application_kind = 'A' AND
    ipc IS NOT NULL AND
    country_code IS NOT NULL AND
    publication_date > 20180000
ORDER BY RAND()
LIMIT 1000000;
  """

This query was around 20GB

In [ ]:
bq_assistant.estimate_query_size(query2018)

Queries were run to pull 2 datasets (one for analysis part of project, one for modelling part of project) from the total Google Patent Dataset

In [ ]:
## pulling the data
response = patents.query_to_pandas_safe(query2018, max_gb_scanned=20)
response.head(10)

In [ ]:
response2 = patents.query_to_pandas_safe(query_patent_text, max_gb_scanned=250)
response2.head(10)